# Oscillation tests

This notebook is intended to generate dark images in the region where the oscillations are the worst.\
Craig Lage - 24-Jul25


In [ ]:
import asyncio
import time
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
# Disable the overslew.
atcs._overslew_az

In [ ]:
atcs._overslew_az = True

In [ ]:
atcs._overslew_az

## Enable LATISS and ATCS

### Bias verification

In [ ]:
for i in range(5):
    await latiss.take_bias(nbias=1)
    await asyncio.sleep(2.0)

In [ ]:
await atcs.disable_dome_following()

## First try several locations, taking a 30 second image at each location.
## We did not see oscillations this way.

In [ ]:
# Move to starting location
coord=atcs.radec_from_azel(az=230, el=45)
await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
await latiss.take_darks(30.0, 1)

In [ ]:
# Move to starting location
coord=atcs.radec_from_azel(az=220, el=45)
await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
await latiss.take_darks(30.0, 1)

In [ ]:
await latiss.take_darks(30.0, 1)

In [ ]:
# Move to starting location
coord=atcs.radec_from_azel(az=210, el=45)
await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
await latiss.take_darks(30.0, 1)

In [ ]:
await latiss.take_darks(30.0, 1)

# The loop below was very successful at reproducing the oscillations.  We always saw medium size oscillations at Az=200 and larger oscillations at Az=185.

In [ ]:
# Move to starting location
coord=atcs.radec_from_azel(az=270, el=45)
await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)

az = 270.0
await latiss.take_darks(30.0, 1)
for i in range(20):
    az -= 5.0
    coord=atcs.radec_from_azel(az=az, el=45)
    await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
    await latiss.take_darks(30.0, 1)